# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [22]:
import requests
from key import fred_key
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


base_url = "https://api.stlouisfed.org/fred/"
obs_endpoint = "series/observations"

start_date = "1950-01-01"
end_date = "2019-12-31"


def create_series_dict(name, series_id):
    return {
        'name': name,
        'series_id': series_id,
        'api_key': fred_key,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
        'frequency': 'q',
        'units': 'lin'
    }

series_list = [
    ('GDP', 'GDPC1'),
    ('W', 'LES1252881600Q'),
    ('Labour', 'PAYEMS'),
    
]

request_parameters = [create_series_dict(name, series_id) for name, series_id in series_list]

In [23]:
def fetch_data(parameter):
    response = requests.get(base_url + obs_endpoint, params=parameter)
    if response.status_code == 200:
        res_data = response.json()
        obs_data = pd.DataFrame(res_data['observations'])
        obs_data['date'] = pd.to_datetime(obs_data['date'])
        obs_data.set_index('date', inplace=True)
        obs_data = obs_data.drop(['realtime_start', 'realtime_end'], axis=1)

        # Convert 'value' to numeric, coercing errors to NaN
        obs_data['value'] = pd.to_numeric(obs_data['value'], errors='coerce')

        obs_data.rename(columns={'value': parameter['name']}, inplace=True)
        return obs_data
    else:
        print('Failed to retrieve data. Status code:', response.status_code)
        return pd.DataFrame()  # Return an empty DataFrame on failure
    
    
dataframe = pd.DataFrame()
# concatenate the data
for parameter in request_parameters:
    df = fetch_data(parameter)
    if dataframe.empty:
        dataframe = df
    else:
        dataframe = pd.concat([dataframe, df], axis=1)

In [24]:
dataframe = dataframe.dropna()
dataframe

,GDP,W,Labour
date,,,
1979-01-01,7238.727,335.0,89114
1979-04-01,7246.454,335.0,89771
1979-07-01,7300.281,330.0,90281
1979-10-01,7318.535,326.0,90575
1980-01-01,7341.557,321.0,90892
...,...,...,...
2018-10-01,20304.874,355.0,149650
2019-01-01,20415.150,355.0,150141
2019-04-01,20584.528,358.0,150695


In [27]:
dataset_hp  = pd.DataFrame()
for column in dataframe.columns:
    cycle, trend = sm.tsa.filters.hpfilter(dataframe[column], lamb=1600)
    dataset_hp[column] = trend
dataset_hp

,GDP,W,Labour
date,,,
1979-01-01,7104.501764,326.264388,89283.720347
1979-04-01,7130.622359,325.041845,89387.118836
1979-07-01,7156.826844,323.824761,89490.411250
1979-10-01,7183.271506,322.624821,89593.731439
1980-01-01,7210.202289,321.457567,89697.707372
...,...,...,...
2018-10-01,20348.548133,355.154892,149701.167514
2019-01-01,20475.838973,356.290300,150302.278492
2019-04-01,20603.398910,357.432107,150901.957005


In [28]:
corr = dataset_hp.corr()
corr

,GDP,W,Labour
GDP,1.000000,0.903310,0.977847
W,0.903310,1.000000,0.848103
Labour,0.977847,0.848103,1.000000
